<a href="https://colab.research.google.com/github/KaihangZhao/DL_Notebook_Warehouse/blob/main/Bert_emb_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer

df7000 = pd.read_csv('IGExport_hannibalburess_7007_followers.csv').drop_duplicates(subset='User ID').reset_index(drop = True)
df_candidate = df7000.drop_duplicates().reset_index(drop = True)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
text = df_candidate['Biography']
#marked_text = "[CLS] " + text + " [SEP]"

marked_text = text.fillna("")
marked_text = text.astype(str).values.tolist()
# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer(marked_text)

# Print out the tokens.
print(tokenized_text)

In [36]:
segments_ids = [1] * len(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
segments_tensors

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
# with torch.no_grad():
#   emb = []
#   for i in range(len(marked_text)):
#     tokens_tensor = torch.tensor(tokenized_text['input_ids'])
#     segments_tensors = torch.tensor(tokenized_text['attention_mask'])
#     outputs = model(tokens_tensor, segments_tensors)
#     # Evaluating the model will return a different number of objects based on 
#     # how it's configured in the `from_pretrained` call earlier. In this case, 
#     # becase we set `output_hidden_states = True`, the third item will be the 
#     # hidden states from all layers. See the documentation for more details:
#     # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
#     hidden_states = outputs[2][-1].sum(0)
#     emb.append(hidden_states)

In [40]:
with torch.no_grad():
  emb = []
  for i in range(len(marked_text)):
    tokens_tensor = torch.tensor(tokenized_text['input_ids'][i]).unsqueeze(0)
    segments_tensors = torch.tensor(tokenized_text['attention_mask'][i]).unsqueeze(0)
    outputs = model(tokens_tensor, segments_tensors)
    # Evaluating the model will return a different number of objects based on 
    # how it's configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2][-1].mean(1)
    emb.append(hidden_states)

In [55]:
df = pd.DataFrame()
for i in emb:
  df = pd.concat([df, pd.DataFrame(i.numpy())])
df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.147346,-0.120473,-0.143501,-0.050273,-0.246980,-0.174424,0.217534,-0.021909,0.250342,-0.369516,...,0.510520,-0.168481,-0.157159,-0.270925,-0.184986,-0.467402,0.170688,-0.064285,-0.171124,-0.133782
0,0.323784,0.080362,0.302904,0.037508,0.254869,-0.137709,0.258920,0.325204,-0.236397,-0.317432,...,0.295823,0.137647,-0.218443,-0.402156,0.111328,-0.091758,-0.275531,-0.215452,0.002792,0.041189
0,0.147346,-0.120473,-0.143501,-0.050273,-0.246980,-0.174424,0.217534,-0.021909,0.250342,-0.369516,...,0.510520,-0.168481,-0.157159,-0.270925,-0.184986,-0.467402,0.170688,-0.064285,-0.171124,-0.133782
0,0.147346,-0.120473,-0.143501,-0.050273,-0.246980,-0.174424,0.217534,-0.021909,0.250342,-0.369516,...,0.510520,-0.168481,-0.157159,-0.270925,-0.184986,-0.467402,0.170688,-0.064285,-0.171124,-0.133782
0,-0.520562,-0.264124,-0.218240,-0.143760,-0.367288,-0.036141,0.358598,-0.021013,-0.080577,-0.001565,...,0.521648,-0.267559,0.278446,-0.246969,-0.038839,-0.428954,0.007199,0.133869,0.265998,-0.562217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.712374,-0.101119,0.389781,0.037115,0.104205,-0.136739,0.370734,0.756381,-0.155270,-0.235989,...,0.269902,0.107363,-0.499069,-0.645313,0.012817,-0.138702,-0.197414,-0.239744,0.273811,0.320092
0,0.237853,0.114790,0.659308,-0.277845,0.773006,0.066712,-0.014950,0.321674,-0.329409,0.059084,...,0.148513,0.267546,0.120456,-0.701267,0.180478,-0.012314,-0.102265,-0.312779,-0.329412,-0.009076
0,-0.134046,-0.250342,-0.343378,-0.206649,0.160972,-0.309760,0.453729,0.093569,0.019891,-0.187317,...,0.018644,0.007957,0.081386,-0.283665,-0.001257,-0.312304,-0.085550,-0.113256,-0.232458,0.047850
0,0.050293,0.183082,0.196702,-0.434516,0.154140,0.070074,0.220508,0.394022,0.134865,0.111115,...,0.339520,-0.008137,-0.023215,-0.415962,-0.014262,0.277234,0.103639,-0.368668,0.122899,0.406845


In [58]:
df.reset_index(drop=True).to_csv('7007_emb.csv', index=False)